<a href="https://colab.research.google.com/github/yhp2205/SQL/blob/main/ch_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 7강 - 하나의 테이블에 대한 조작

In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## 그룹의 특징 잡기
새로운 테이블을 사용하여 집약함수를 사용하는 방법을 알아보겠습니다.

In [3]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.review;
CREATE TABLE sqldata.review (
    user_id    string
  , product_id string
  , score      numeric
);

INSERT INTO sqldata.review
VALUES
    ('U001', 'A001', 4.0)
  , ('U001', 'A002', 5.0)
  , ('U001', 'A003', 5.0)
  , ('U002', 'A001', 3.0)
  , ('U002', 'A002', 3.0)
  , ('U002', 'A003', 4.0)
  , ('U003', 'A001', 5.0)
  , ('U003', 'A002', 4.0)
  , ('U003', 'A003', 4.0)
;


""


다음과 같이 review 테이블을 생성했습니다.  
COUNT함수는 지정 컬럼의 record 수를 출력하는 함수입니다.  
컬럼 앞에 DISTINCT라고 지정하면 중복을 제외한 수를 출력합니다. 

In [7]:
%%bigquery --project mygcpproject-340112
  -- 7.1 집약 함수를 사용해서 테이블 전체의 특징량을 계산하는 쿼리
SELECT
    COUNT(*) AS total_count
  , COUNT(DISTINCT user_id) AS user_count
  , COUNT(DISTINCT product_id) AS product_count
  , SUM(score) AS sum
  , AVG(score) AS avg
  , MAX(score) AS max
  , MIN(score) AS min
FROM
  sqldata.review
;

,total_count,user_count,product_count,sum,avg,max,min
0,9,3,3,37,4.111111111,5,3
